# 将上海市旅游数据月分为日

## 1.数据准备
原则：将之前预测的2019全年数据分配作为上海市旅游入境疫情前全年数据日分配，2020-2024依照异常数据处理原则每年一分配

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import torch

# 创建数据库引擎
database_url = "sqlite:///../data/data.sqlite"
engine = create_engine(database_url)

imputed_data_query = f"SELECT * FROM hk_customs_daily_imputed"
df = pd.read_sql_query(imputed_data_query, engine, index_col='date',parse_dates=['date'])

In [2]:
# 计算每个月的总和
monthly_sums = df.resample('ME').sum()
# 初始化一个新的DataFrame来存储每日比例数据
daily_ratio = pd.DataFrame(index=df.index)
# 对于df中的每一列，计算每日比例
for column in df.columns:
    # 对于每个月，计算该月每一天的值占该月总和的比例
    daily_ratio[column] = df[column] / df[column].resample('ME').transform('sum')

In [3]:
regular_slice_start = '2019-01-01'
regular_slice_end = '2019-12-31'
special_slice_start = '2020-01-01'
special_slice_end = '2024-01-31'

regular_range = pd.date_range(start=pd.to_datetime(regular_slice_start), end=pd.to_datetime(regular_slice_end))
epd_range = pd.date_range(start=pd.to_datetime(special_slice_start), end=pd.to_datetime(special_slice_end))

regular_ratio = daily_ratio[regular_slice_start:regular_slice_end]
special_ratio = daily_ratio[special_slice_start:special_slice_end]

# 数据插值
疫情期间数据直接插值

In [4]:
raw_data_query = f"SELECT * FROM sh_customs"
sh_monthly = pd.read_sql_query(raw_data_query, engine, index_col='date',parse_dates=['date'])

# 确定最后一个月的最后一天
last_month = sh_monthly.index.max().to_period('M').to_timestamp('M')

# 生成到最后一个月最后一天的完整日期范围，避免最后一个月没有值
full_date_range = pd.date_range(start=sh_monthly.index.min(), end=last_month, freq='D')

# 重采样并前向填充
sh_daily_fake = sh_monthly.reindex(full_date_range).ffill()

## 疫情期间数据插值

In [5]:
# 为每个 DataFrame 创建年份和月份的列

sh_daily_fake_epd = sh_daily_fake.loc[epd_range]

# 使用pandas广播机制
sh_daily_epd = sh_daily_fake_epd.multiply(special_ratio['CN_airport_departure'],axis=0).astype('int')
# sh_daily = sh_daily_fake.loc[epd_range] * special_ratio['CN_airport_departure']